# Review 정제, 직접 정의한 feature로 벡터화, cosine distance 계산
1. review를 okt(구 트위터 형태소 분석기)와 kkma(꼬꼬마 형태소 분석기)를 활용해서 전처리한다. morphy함수 사용.
2. feature를 list 형식으로 정의한다. 유의어 계속 추가해야함.
3. morphy한 형태소에서 feature를 찾아내서 정규화
4. 정규화 된 벡터끼리 코사인 거리 측정

2019_01_16

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import *
import numpy as np
from scipy import spatial
okt = Okt()

## feature와 stopwords

In [11]:
#feature lists : 각 feature의 값을 결정할 단어(유의어) 모음
sad = set(['슬프','초라','비극적',])
horror = set(['공포','무시','섬뜩하','불안'])
cruel = set(['잔인','잔혹'])
scale = set(['거대','화려','넘치','장엄','현란','압도적'])
family = set(['성장','소년','소녀','수련','가족','일족','집','혈통','종족','가구','친척','혈연','관계','자손','가정','아들','딸', '아버지', ])
futuristic = set(['우주','미래',])
love = set(['로맨스','사랑','연인',])
hero = set(['영웅',''])

##indexing 으로 feature를 전달하는 방법?
#featureList = ['sad','horror', 'cruel', 'scale']

In [12]:
#stopword: 2글자 이상 단어 중 필요 없는 단어 계속 추가해서 필터링!
stop = ["있다","면서","다는","은데","특히","있었","동안","면서","을까","해하","어떤","한때","어야","듯이","ㄴ다",'story']

### morphy : 리뷰에서 형용사, 부사만 뽑아낸 후 형태소 분석

In [13]:
def morphy(review):
    '''    
    pos = okt.pos(review)
    st = ""
    for i in pos:
        if((i[1] == 'Adjective' or i[1] == 'Adverb')):
            st += " "
            st += i[0]
    '''
    kkma = Kkma()
    morphs = kkma.morphs(review)
    morphs = [w for w in morphs if ((w not in stop) and (len(w)>1))]
    return morphs

### feature detecter : 
주어진 feature내 유사어를 형태소에서 찾아서 수를 리턴

In [14]:
#feat은 sad, horror 같은 feature 유의어 사전
#mor는 사용할 morphs (리뷰)
def featureDetector(feat, mor):
    cnt = 0
    for w in mor:
        if w in feat:
            cnt+=1
    return cnt

### feature Normalizer:
Detect한 feature를 정규화

In [15]:
def featureNormalization(feature):
    total = 0
    for i in range(len(feature)):total += feature[i]
    for i in range(len(feature)):feature[i] /= total
    return feature

# 예시

In [16]:
review1 = open('data\cine21\스타 워즈 에피소드 2 클론의 습격(2002).txt',encoding='UTF8').read()
review2 = open('data\cine21\스타 워즈  에피소드 1 - 보이지 않는 위험(1999).txt',encoding='UTF8').read()

In [17]:
morphs1 = morphy(review1)
morphs2 = morphy(review2)

In [18]:
#전처리 결과
print(morphs1)
print('NEXT MOVIE')
print(morphs2)

['Story', '10', '투인', '노예', '소년', '스카이', '헤이', '크리스', '다이', '기사', '오비완', '노비', '이완', '그리', '수련', '제자', '성장', '나부', '행성', '여왕', '거치', '은하계', '상원', '의원', '아미', 'ㄹ라', '포트', '공화국', '체제', '반대', '분리', '주의', '세력', '으로', '부터', '암살', '위협', '위험', '처하', '아미', '달라', '경호', '사이', '금지', '사랑', '싹트', '한편', '분리', '주의자', '음모', '공화국', '의장', '타인', '야심', '충돌', '클론', '전쟁', '임박', 'Review', '스타워즈', '현상', '계속', '영화', '언론', '조지', '루카스', '손아귀', '옛날', '옛적', '은하계', '조금', '빨리', '조금', '많이', '엿보', '위하', '조급증', '그리고', '과거', '보다', '미래', '원인', '보다', '결과', '먼저', '누설', '기이', '거대', '스페이스', '오페라', '퍼즐', '맞추', '향수', '스타워즈', '에피소드', '클론', '습격', '역시', '기본적', '으로', '그렇', '스타워즈', '위하', '영화', '번째', '에피소드', '훗날', '굴복', '비운', '영웅', '스카이워커', '성장', '갈등', '아미', '달라', '사랑', '빠지', '비극적', '모멘트', '초점', '맞추', '소년', '스승', '오비', '완이', '자신', '재능', '시기', 'ㄴ다고', '만큼', '많이', '어머니', '구하', '못하', '죄책감', '세상', '대하', '분노', '증오', '금지', '사랑', '빠져들', '무모', '열정', '으로', '닥치', '어둡', '운명', '예비', '대목', '요다', '루크', '스카이워커', '걸치', '다이', '계보', '완성', '타인', '세력', 

In [19]:
review3 = open('data\cine21\곡성(2015).txt',encoding='UTF8').read()
morphs3 = morphy(review3)
print(morphs3)

['곡성', '이러', '홍진', '감독', '재능', '경탄', '밖에', '미디어', '황진', '편파', '평론', '영화', '본문', '영화', '내용', '누설', '습니다', '영화', '관람', '바라', 'ㅂ니다', '곡성', '암시', '반전', '통하', '감독', '설치', '관객', '빠뜨리', '영화', '감독', '통제', '트릭', '아트', '즐기', '관객', '함께', '게임', '해당', '장르', '법칙', '따르', '가다', '뒤틀', '형식', '취하', '지만', '그렇', '다고', '그것', '전부', '영화', '아니', '종교', '대하', '가장', '본질적', '질문', '던지', '인간', '존재론', '풀어놓', '혹자', '영화', '전개', '정직', '다고', '불만', '표출', '참고', '일차원', '현혹', '자기', '모순', '아이즈', '곡성', '홀로', '유감', '국제', '신문', '일종', '변칙', '플레이', '라는', '하지만', '그렇', '감독', '플레이', '시작', '너무', '정직', '자신', '보이', '으로', '전개', '영화', '주제', '형식', '대하', '결정적', '힌트', '알려주', '첫째', '복음', '24', '구절', '영화', '주제', '말하', '으로', '영화', '성경', '구절', '영화', '마지막', '아예', '재구성', '어서', '보여주', '낚시', '일본인', '영화', '낚시', '일본인', '미끼', '끼우', '장면', '클로즈업', '어서', '보여주', '낚시', '미끼', '대하', '광의', '대사', '맞물리', '영화', '서사', '러스트', '으로', '밝히', '화투', '바둑', '으로', '시작하', 'ㄴ데', '이렇', '혹시', '너무', '어렵', 'ㄹ까', '중간', '힌트', '화면', '나가', '가령', '도시', '영화', '오형', '사가', '성인', '기저귀', '

In [20]:
review4 = open('data\cine21\캐롤(2015).txt',encoding='UTF8').read()
morphs4 = morphy(review4)
print(morphs4)

['1950', '년대', '뉴욕', '사진가', '꿈꾸', '맨해튼', '백화점', '점원', '테레즈', '마라', '멀리서', '캐롤', '케이트', '첫눈', '그녀', '에게', '이끌리', '테레즈', '매장', '캐롤', '장갑', '사이', '점차', '가깝', '이혼', '소송', '중인', '남편', '미지근', '관계', '애인', '캐롤', '테레즈', '서로', '허전', '마음', '헤아리', '사랑', '키우', '나가', '토드', '헤인즈', '로맨스', '프롬', '2002', '드레드', '피어스', '2011', '과거', '배경', '으로', '곤경', '빠지', '여자', '사랑', '차분', '따라가', '작품', '번째', '장편', '캐롤', '역시', '내놓', '멜로드라마', '특징', '드러나', '한편', '주인공', '사람', '으로', '음에도', '영화', '감싸', '감정', '더욱', '절제', '사랑이', '불어나', '기쁨', '에서', '슬쩍', '미소', '흘리', '갑자기', '들이닥치', '주변', '방해', '눈물', '다만', '영화', '사랑', '징후', '더없이', '풍부', '특정', '사건', '경유', '미세', '변하', '테레즈', '캐롤', '서로', '대하', '마음', '케이트', '블란쳇', '마라', '섬세', '연기', '통하', '완벽', '구현', '사소', '표정', '이나', '제스처', '하나하나', '놓치', '아쉽', '명연', '캐롤', '그저', '소품', '으로서', '마음', '빼앗', '기에', '충분', '만들', '의상', '베커', '지휘', '프로덕션', '디자인', '프롬', '이후', '토드', '헤인즈', '꾸준히', '작업', '촬영', '감독', '에드워드', '라흐만', '카메라', '에서', '발하', '스포일러', '습니다', '토드', '헤인즈', '캐롤', '2015', '도입부', '에서', '데이', '비드', 

In [21]:
total = 0
feature4 = []
feature4.append(featureDetector(sad, morphs4))
feature4.append(featureDetector(horror, morphs4))
feature4.append(featureDetector(cruel, morphs4))
feature4.append(featureDetector(scale, morphs4))
feature4.append(featureDetector(love, morphs4))
feature4 = featureNormalization(feature4)
print(feature4)
print(featureDetector(love, morphs4))

[0.0, 0.15384615384615385, 0.0, 0.11538461538461539, 0.7307692307692307]
19


In [8]:
total = 0
feature3 = []
feature3.append(featureDetector(sad, morphs3))
feature3.append(featureDetector(horror, morphs3))
feature3.append(featureDetector(cruel, morphs3))
feature3.append(featureDetector(scale, morphs3))
feature3 = featureNormalization(feature3)
print(feature3)
print(featureDetector(horror, morphs3))

NameError: name 'sad' is not defined

In [9]:
total = 0
feature1 = []
feature1.append(featureDetector(sad, morphs1))
feature1.append(featureDetector(horror, morphs1))
feature1.append(featureDetector(cruel, morphs1))
feature1.append(featureDetector(scale, morphs1))
feature1 = featureNormalization(feature1)

feature2 = []
feature2.append(featureDetector(sad, morphs2))
feature2.append(featureDetector(horror, morphs2))
feature2.append(featureDetector(cruel, morphs2))
feature2.append(featureDetector(scale, morphs2))
feature2 = featureNormalization(feature2)


print(feature1)
print(feature2)

NameError: name 'sad' is not defined

<b>Cosine distance 구하는 법:

In [11]:
result = 1 - spatial.distance.cosine(feature1, feature2)
result

0.9970544855015816

In [15]:
result2 = 1 - spatial.distance.cosine(feature1, feature3)
result2

0.3651483716701107